In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, urllib.request, requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import csv
import re
import random

In [98]:
# Inisialisasi driver dengan service
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [100]:
#setting username dan password
USERNAME = 'laros.woodart'  # Ganti dengan akun kamu
PASSWORD = 'bismillahkbmi'  # Ganti dengan password akun kamu
TARGET_PROFILE = 'jokowi'            # Target akun yang mau diambil komentar

In [102]:
#buka instagram
driver.get('https://www.instagram.com')

In [104]:
# Tunggu field username muncul
wait = WebDriverWait(driver, 15)
username_input = wait.until(EC.presence_of_element_located((By.NAME, 'username')))
password_input = wait.until(EC.presence_of_element_located((By.NAME, 'password')))

In [106]:
# Isi username dan password
username_input.send_keys(USERNAME)
password_input.send_keys(PASSWORD)
password_input.send_keys(Keys.RETURN)

time.sleep(5)  # Tunggu proses login

In [108]:
### 2. Buka profil target
driver.get(f'https://www.instagram.com/{TARGET_PROFILE}/')
time.sleep(5)

# Scroll halaman untuk load semua postingan
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 5))

In [110]:
# Scroll sedikit dulu biar postingan mulai kebuka
for _ in range(1):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 5))
links = []

# Scroll halaman sampai dapat 300 postingan
while True:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    for a in soup.find_all('a', href=True):
        if '/p/' in a['href']:
            links.append("https://www.instagram.com" + a['href'])
    
    links = list(set(links))  # Remove duplicate

    print(f"Total postingan ditemukan: {len(links)}")

    if len(links) >= 300:
        break
    
    # Scroll ke bawah untuk load lebih banyak postingan
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 5))

print(f"Total link postingan terkumpul: {len(links)}")

Total postingan ditemukan: 0
Total postingan ditemukan: 1
Total postingan ditemukan: 4
Total postingan ditemukan: 8
Total postingan ditemukan: 12
Total postingan ditemukan: 17
Total postingan ditemukan: 26
Total postingan ditemukan: 36
Total postingan ditemukan: 44
Total postingan ditemukan: 54
Total postingan ditemukan: 65
Total postingan ditemukan: 72
Total postingan ditemukan: 82
Total postingan ditemukan: 92
Total postingan ditemukan: 103
Total postingan ditemukan: 113
Total postingan ditemukan: 120
Total postingan ditemukan: 128
Total postingan ditemukan: 139
Total postingan ditemukan: 148
Total postingan ditemukan: 157
Total postingan ditemukan: 166
Total postingan ditemukan: 177
Total postingan ditemukan: 184
Total postingan ditemukan: 192
Total postingan ditemukan: 200
Total postingan ditemukan: 210
Total postingan ditemukan: 219
Total postingan ditemukan: 225
Total postingan ditemukan: 231
Total postingan ditemukan: 239
Total postingan ditemukan: 248
Total postingan ditemukan:

In [76]:
# --- Fungsi hapus emoji ---
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emotikon
        "\U0001F300-\U0001F5FF"  # simbol & pictographs
        "\U0001F680-\U0001F6FF"  # transport
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002500-\U00002BEF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"
        "\u3030"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# --- Setting ---
total_comments_target = 10000

comments = []

for post_link in tqdm(links):
    driver.get(post_link)
    time.sleep(5)

    # Klik "View more comments" berkali-kali
    while True:
        try:
            load_more = driver.find_element(By.XPATH, '//span[contains(text(), "View more comments") or contains(text(), "Load more comments") or contains(text(),"View all comments")]')
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(2)
        except:
            break  # Tidak ada tombol lagi

    # Scroll supaya semua komentar keluar
    scroll_pause_time = 5
    scroll_count = 0
    max_scrolls = 50

    last_height = driver.execute_script("return document.body.scrollHeight")
    while scroll_count < max_scrolls:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        last_height = new_height
        scroll_count += 1

    # --- Ambil komentar (setelah selesai scroll) ---
    try:
        comment_elements = driver.find_elements(By.XPATH, '//span[contains(@class, "ap3a") and contains(@class, "_aaco") and contains(@class, "_aacu") and contains(@class, "_aacx") and contains(@class, "_aad7") and contains(@class, "_aade")]')
        post_comment_count = 0

        for elem in comment_elements:
            raw_comment = elem.text.strip()
            clean_comment = remove_emojis(raw_comment).strip()

            if clean_comment:
                comments.append(clean_comment)
                post_comment_count += 1

        print(f"Komentar dari postingan ini: {post_comment_count}, Total komentar terkumpul: {len(comments)}")

    except Exception as e:
        print(f"Gagal ambil komentar di {post_link}, error: {e}")

    if len(comments) >= total_comments_target:
        break

print(f"\nTotal semua komentar terkumpul: {len(comments)}")

  0%|          | 1/304 [00:11<1:00:05, 11.90s/it]

Komentar dari postingan ini: 0, Total komentar terkumpul: 0


  1%|          | 2/304 [00:23<57:55, 11.51s/it]  

Komentar dari postingan ini: 0, Total komentar terkumpul: 0


  1%|          | 3/304 [00:34<56:27, 11.25s/it]

...

Komentar dari postingan ini: 14, Total komentar terkumpul: 3863


100%|█████████▉| 303/304 [57:27<00:11, 11.52s/it]

Komentar dari postingan ini: 13, Total komentar terkumpul: 3876


100%|██████████| 304/304 [57:38<00:00, 11.38s/it]

Komentar dari postingan ini: 14, Total komentar terkumpul: 3890

Total semua komentar terkumpul: 3890


In [78]:
# --- Simpan ke CSV ---
with open('komentar.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Komentar'])
    for comment in comments:
        writer.writerow([comment])

# Tutup driver
driver.quit()

In [80]:
data=pd.read_csv("komentar.csv")
data.head(100)

,Komentar
0,Tetap ada oyen
1,Selamat Tahun Baru Islam 1446 H! Semoga Allah ...
2,Masya allah
3,Tim yg cari kucing oyen ---------->
4,selamat tahun Baru Islam
...,...
95,Saya juga mau berkunjung boleh nggak pak?
96,Lg santai kawan
97,Sehat selalu bapak jokowi
98,Sehat sll bpk
